In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [2]:
def read_dat_file(file_path):
    """"Função para a leitura das instâncias geradas"""
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # 1. Lendo quantidade de itens e períodos
    items, periods = map(int, lines[0].split())

    # 2. Lendo número de plantas
    num_plants = int(lines[1].strip())

    # 3. Lendo capacidades das plantas
    capacities = [int(lines[i + 2].strip()) for i in range(num_plants)]
    capacities = np.tile(capacities, (periods, 1)).T  # Repete as capacidades ao longo dos períodos (deixar na forma j, t)

    # 4. Lendo a matriz de produção (tempo de produção, tempo de setup, custo de setup, custo de produção)
    production_data = []
    start_line = 2 + num_plants
    production_time = np.zeros((items, num_plants))  # Inicializar listas para armazenar separadamente os tempos e custos
    setup_time = np.zeros((items, num_plants))
    setup_cost = np.zeros((items, num_plants))
    production_cost = np.zeros((items, num_plants))
    for i in range(num_plants * items):  # Preencher as matrizes com os dados lidos
        plant = i // items  # Determina a planta
        item = i % items    # Determina o item
        # Extrair os dados de cada linha
        prod_time, set_time, set_cost, prod_cost = map(float, lines[start_line + i].split())
        production_time[item, plant] = prod_time  # Preencher as respectivas matrizes
        setup_time[item, plant] = set_time
        setup_cost[item, plant] = set_cost
        production_cost[item, plant] = prod_cost

    # 5. Lendo os custos de inventário
    inventory_costs_line = start_line + num_plants * items
    inventory_costs = list(map(float, lines[inventory_costs_line].split()))  # Lê todos os valores de inventory_costs como uma única lista
    inventory_costs = np.array(inventory_costs).reshape(num_plants, -1)  # Divide a lista de custos de inventário por planta
    inventory_costs = inventory_costs.T  # Deixa na forma (i, j)

    # 6. Lendo a matriz de demanda (12 linhas)
    demand_matrix = []
    demand_start_line = inventory_costs_line + 1
    
    # Leitura inicial das demandas
    for i in range(periods):  # Lê as linhas de demandas para os períodos
        demands = list(map(int, lines[demand_start_line + i].split()))
        demand_matrix.append(demands)
    
    # Agora vamos dividir os valores de cada linha combinada entre as plantas
    final_demand_matrix = []
    for demands in demand_matrix:
        period_demand = []
        for j in range(num_plants):
            # Divide a demanda combinada por planta, assumindo que cada planta tem o mesmo número de itens
            plant_demand = demands[j*items:(j+1)*items]
            period_demand.append(plant_demand)
        final_demand_matrix.append(period_demand)
    
    # Transpor a matriz de demanda para o formato correto (itens, plantas, períodos)
    final_demand_matrix = np.array(final_demand_matrix)
    final_demand_matrix = np.transpose(final_demand_matrix, (2, 1, 0))  # Converte para o formato (itens, plantas, períodos)

    # 7. Reading transfer costs directly from the document as a matrix
    transfer_cost_matrix = []
    transfer_cost_line = demand_start_line + periods

    # Read the matrix of transfer costs line by line
    while transfer_cost_line < len(lines):
        line = lines[transfer_cost_line].strip()
        if line:
            # Split the line into individual cost values and convert them to float
            row = [float(value) for value in line.split()]
            transfer_cost_matrix.append(row)
        transfer_cost_line += 1

    # Convert to a numpy array (optional, if you want to work with numpy for matrix operations)
    transfer_costs = np.array(transfer_cost_matrix)

    return {"items": items,
            "periods": periods,
            "num_plants": num_plants,
            "capacities": capacities,
            "production_time": production_time,
            "setup_time": setup_time,
            "setup_cost": setup_cost,  
            "production_cost": production_cost,
            "inventory_costs": inventory_costs,
            "demand_matrix": final_demand_matrix,
            "transfer_costs": transfer_costs}

In [3]:
folder_path = '../instancias/multi_plant_instances/'
instancia = 'NBB03_12_2_10.dat'

In [4]:
data = read_dat_file(folder_path + instancia)

# Modelo
m = gp.Model(instancia.replace('.dat', ''))

# Conjuntos
# Produtos (i)
I = np.array([_ for _ in range(data['items'])])
# Plantas (j)
J = np.array([_ for _ in range(data['num_plants'])])
# Períodos (t)
T = np.array([_ for _ in range(data['periods'])])

# Parâmetros
# Demanda (i, j, t)
d = np.array(data['demand_matrix'])
# Capacidade (j, t)
cap = np.array(data['capacities'])
# Tempo de produção (i, j)
b = np.array(data['production_time'])
# Tempo de setup (i, j)
f = np.array(data['setup_time'])
# Custo de produção (i, j)
c = np.array(data['production_cost'])
# Custo de setup (i, j)
s = np.array(data['setup_cost'])
# Custo de transporte (j, k)
r = np.array(data['transfer_costs'])
# Custo de estoque (i, j)
h = np.array(data['inventory_costs'])

# Variáveis de decisão
# Quantidade produzida (i, j, t)
X = m.addVars(I, J, T, vtype=GRB.CONTINUOUS, name='X')
# Quantidade estocada (i, j, t)
Q = m.addVars(I, J, T, vtype=GRB.CONTINUOUS, name='Q')
# # Quantidade transportada (i, j, k(um outro j), t)
W = m.addVars(I, J, J, T, vtype=GRB.CONTINUOUS, name='W')
# # Variável de setup (binária -> relaxada) (i, j, t)
Z = m.addVars(I, J, T, lb=0, ub=1, vtype=GRB.BINARY, name='Z')

# Função objetivo
expr_objetivo = sum(sum(sum(c[i, j] * X[i, j, t] + h[i, j] * Q[i, j, t] + s[i, j] * Z[i, j, t] + 
                        sum(r[j, k] * W[i, j, k, t] for k in J if k != j) for t in T) for j in J) for i in I)
m.setObjective(expr_objetivo, sense=GRB.MINIMIZE)

# Restrições
# Balanço de estoque (revisar comportamento)
    # Período inicial
m.addConstrs((Q[i, j, t] == X[i, j, t] - sum(W[i, j, k, t] for k in J if k != j) + sum(W[i, l, j, t] for l in J if l != j) - d[i, j, t] for i in I for j in J for t in T if t == 0),
            name='restricao_balanco_estoque')
    # Demais períodos
m.addConstrs((Q[i, j, t] == Q[i, j, t-1] + X[i, j, t] - sum(W[i, j, k, t] for k in J if k != j) + sum(W[i, l, j, t] for l in J if l != j) - d[i, j, t] for i in I for j in J for t in T if t > 0),
            name='restricao_balanco_estoque')

# Restrição que obriga setup (validar o range do r)
m.addConstrs((X[i, j, t] <= min((cap[j, t] - f[i, j]) / b[i, j], sum(sum(d[i, k, r] for r in range(t, T[-1] + 1)) for k in J)) * Z[i, j, t] for i in I for j in J for t in T)
            , name='restricao_setup')

# Restrição de capacidade
m.addConstrs((sum(b[i, j] * X[i, j, t] + f[i, j] * Z[i, j, t] for i in I) <= cap[j, t] for j in J for t in T)
            , name='restricao_capacidade');

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-13


In [5]:
m.update()
print(m)
print(m.Fingerprint)

<gurobi.Model MIP instance NBB03_12_2_10: 504 constrs, 1200 vars, Parameter changes: Username=(user-defined)>
829360891


In [6]:
m.Params.OutputFlag = 0  # Desativa a exibição de logs
m.Params.SolutionLimit = 1  # Configurar o limite de solução para parar após encontrar a N_ésima solução factível  
m.optimize()
m.Params.SolutionLimit = 2_000_000_000  # Retorna para o default

In [7]:
# Valor do objetivo da primeira solução
m.ObjVal

62972.66623077676

In [ ]:
# Extrair a solução inicial de referência
reference_solution = {key: Z[key].x for key in Z.keys()}  # _x
best_solution = reference_solution  # x*
best_obj_val = m.objVal if m.status == GRB.OPTIMAL else float('inf')
# Parâmetro de vizinhança
K = 20
node = 1
# Adicionar a restrição Δ(x, x*) ≤ K
delta_expr = gp.quicksum((1 - best_solution[key]) * Z[key] + best_solution[key] * (1 - Z[key]) for key in Z.keys())
m.addConstr(delta_expr <= K, "local_branching")
print(f'Explorando o lado esquerdo no nó {node + 1}')
# Loop principal do Algoritmo de Local Branching
while True:
    node += 1
    # Resolver o subproblema resultante  
    m.optimize()

    if m.SolCount > 0 and m.objVal < best_obj_val:
        # Restringir para o espaço da melhor solução obtida (FO <= FO X*)
        # Atualizar a melhor solução
        best_obj_val = m.objVal
        best_solution = {key: Z[key].x for key in Z.keys()}      
        if node % 2 == 0:  # Após exploração do branch esquerdo (nós pares) vai para uma exploração à direita
            # Remover a restrição anterior e adicionar Δ(x, x*) ≥ K + 1
            m.remove(m.getConstrByName("local_branching"))
            m.addConstr(delta_expr >= K + 1, "local_branching")
            print(f'Explorando o lado direito no nó {node + 1}')
        else:  # Após exploração do branch direito (nós ímpares) vai para uma exploração à esquerda
            # Remover a restrição anterior e adicionar Δ(x, x*) <= K
            m.remove(m.getConstrByName("local_branching"))
            delta_expr = gp.quicksum((1 - best_solution[key]) * Z[key] + best_solution[key] * (1 - Z[key]) for key in Z.keys())
            m.addConstr(delta_expr <= K, "local_branching")
            print(f'Explorando o lado esquerdo no nó {node + 1}')
    else:  # Se nenhuma solução melhor for encontrada, sair do loop
        m.remove(m.getConstrByName("local_branching"))
        m.addConstr(delta_expr >= K + 1, "local_branching")
        break

# Resolver o subproblema final para tentar encontrar uma melhor solução
print(f'Resolvendo subproblema final (nó {node + 1})')
m.optimize()
if m.SolCount > 0 and m.objVal < best_obj_val:        
    # Atualizar a melhor solução
    best_obj_val = m.objVal
    best_solution = {key: Z[key].x for key in Z.keys()}

Explorando o lado esquerdo no nó 2
Explorando o lado direito no nó 3
Explorando o lado esquerdo no nó 4
Explorando o lado direito no nó 5
Resolvendo subproblema final (nó 6)


In [9]:
# Resultado final
print(f"Melhor valor da função objetivo encontrado: {best_obj_val}")

Melhor valor da função objetivo encontrado: 52283.28508888947
